In [ ]:
import pandas as pd
from google.colab import files
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [ ]:
uploaded = files.upload()

Saving final_data.csv to final_data.csv


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Activity Prediction").getOrCreate()
file_path = "final_data.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)
data.show(5)

+-------+--------+-----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------+-------------------+-------------------+-------------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------

In [ ]:
indexer = StringIndexer(inputCol="Activity", outputCol="label")
data = indexer.fit(data).transform(data)

In [ ]:
data.show(5)

+-------+--------+-----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------+-------------------+-------------------+-------------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------

In [ ]:
feature_columns = data.columns[2:]  # Bỏ cột 'Activity' và 'subject'
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

In [ ]:
data.show(5)

+-------+--------+-----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------+-------------------+-------------------+-------------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------

In [ ]:
from pyspark.ml.feature import StandardScaler


In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
data = scaler.fit(data).transform(data)

In [ ]:
data.show(5)

+-------+--------+-----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------+-------------------+-------------------+-------------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------

In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=100)

In [ ]:
X_train = train_data.select("scaledFeatures")
y_train = train_data.select("label")

X_test = test_data.select("scaledFeatures")
y_test = test_data.select("label")


In [ ]:
print(f"Tập huấn luyện: {train_data.count()} mẫu, Tập kiểm tra: {test_data.count()} mẫu")
print(f"X_train shape: {X_train.count()}, y_train shape: {y_train.count()}")
print(f"X_test shape: {X_test.count()}, y_test shape: {y_test.count()}")


Tập huấn luyện: 8244 mẫu, Tập kiểm tra: 2055 mẫu
X_train shape: 8244, y_train shape: 8244
X_test shape: 2055, y_test shape: 2055


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [ ]:
svc=SVC()

In [ ]:
svc.fit(X_train,y_train)

ValueError: setting an array element with a sequence.

In [ ]:
y_train.show(5)

+-----+
|label|
+-----+
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
+-----+
only showing top 5 rows



In [ ]:
import numpy as np

In [ ]:
X_train_np = np.array(X_train.select('scaledFeatures').rdd.map(lambda row: row[0]).collect())


In [ ]:
y_train_np = np.array(y_train.select('label').rdd.map(lambda row: row[0]).collect())

In [ ]:
X_test_np = np.array(X_test.select('scaledFeatures').rdd.map(lambda row: row[0]).collect())

In [ ]:
y_test_np = np.array(y_test.select('label').rdd.map(lambda row: row[0]).collect())


In [ ]:
svc.fit(X_train_np, y_train_np)

SVC()

In [ ]:
y_pred=svc.predict(X_test_np)

In [ ]:
print('Model accuracy score with default hyperparameters: {0:0.4f}'
. format(accuracy_score(y_test_np, y_pred)))


Model accuracy score with default hyperparameters: 0.9640


In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns=['Predicted'])
y_test_df = pd.DataFrame(y_test_np, columns=['Actual'])

# Concatenate the DataFrames along the columns
combined_df = pd.concat([y_pred_df, y_test_df], axis=1)

In [ ]:
combined_df.head(30)

,Predicted,Actual
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [ ]:
rand_clf=RandomForestClassifier(random_state=5)


In [ ]:
rand_clf.fit(X_train_np,y_train_np)

RandomForestClassifier(random_state=5)

In [ ]:
rand_clf.score(X_test_np,y_test_np)

0.9975669099756691

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


In [ ]:
accuracy = accuracy_score(y_test_np, y_pred)
precision = precision_score(y_test_np, y_pred, average='macro')  # 'macro' phù hợp cho phân loại đa lớp
recall = recall_score(y_test_np, y_pred, average='macro')
f1 = f1_score(y_test_np, y_pred, average='macro')
print('Confusion Matrix:')
print(confusion_matrix(y_test_np, y_pred))
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

class_report = classification_report(y_test_np,y_pred)
print('Classification Report: ')
print(class_report)


Confusion Matrix:
[[362   0   0   0   0   0]
 [  0 356  23   0   0   0]
 [  1  29 325   0   0   0]
 [  0   0   0 338   4   0]
 [  0   0   0   2 347   6]
 [  0   0   0   2   7 253]]
Accuracy: 0.9639902676399027
Precision: 0.965040060522281
Recall: 0.9643707798628037
F1 Score: 0.9646720036758945
Classification Report: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       362
         1.0       0.92      0.94      0.93       379
         2.0       0.93      0.92      0.92       355
         3.0       0.99      0.99      0.99       342
         4.0       0.97      0.98      0.97       355
         5.0       0.98      0.97      0.97       262

    accuracy                           0.96      2055
   macro avg       0.97      0.96      0.96      2055
weighted avg       0.96      0.96      0.96      2055



In [ ]:
y_pred_rf=rand_clf.predict(X_test_np)


In [ ]:
accuracy = accuracy_score(y_test_np, y_pred_rf)
precision = precision_score(y_test_np, y_pred_rf, average='macro')  # 'macro' phù hợp cho phân loại đa lớp
recall = recall_score(y_test_np, y_pred_rf, average='macro')
f1 = f1_score(y_test_np, y_pred_rf, average='macro')
print('Confusion Matrix:')
print(confusion_matrix(y_test_np, y_pred_rf))
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

class_report = classification_report(y_test_np,y_pred_rf)
print('Classification Report: ')
print(class_report)

Confusion Matrix:
[[362   0   0   0   0   0]
 [  0 376   3   0   0   0]
 [  0   1 354   0   0   0]
 [  0   0   0 341   0   1]
 [  0   0   0   0 355   0]
 [  0   0   0   0   0 262]]
Accuracy: 0.9975669099756691
Precision: 0.9975236395654569
Recall: 0.9977239257835068
F1 Score: 0.9976203544054675
Classification Report: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       362
         1.0       1.00      0.99      0.99       379
         2.0       0.99      1.00      0.99       355
         3.0       1.00      1.00      1.00       342
         4.0       1.00      1.00      1.00       355
         5.0       1.00      1.00      1.00       262

    accuracy                           1.00      2055
   macro avg       1.00      1.00      1.00      2055
weighted avg       1.00      1.00      1.00      2055



In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=100)


In [ ]:
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression(featuresCol='scaledFeatures', labelCol='label', maxIter=100, family="multinomial")
log_reg_model = log_reg.fit(train_data)

In [ ]:
predictions = log_reg_model.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy as np
from pyspark.sql.functions import col
from pyspark.sql import Row

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy as np
import pandas as pd

# Tạo RDD chứa nhãn thật và nhãn dự đoán
predictionAndLabels = predictions.select(col("prediction"), col("label")).rdd

# Khởi tạo MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabels)

# Tính toán confusion matrix
confusion_matrix = metrics.confusionMatrix().toArray()
print('Confusion Matrix:')
print(confusion_matrix)

# Lấy số lớp
num_classes = confusion_matrix.shape[0]

# Tính toán Precision, Recall và F1 Score cho từng lớp
report = []

for i in range(num_classes):
    true_positive = confusion_matrix[i, i]
    false_positive = np.sum(confusion_matrix[:, i]) - true_positive
    false_negative = np.sum(confusion_matrix[i, :]) - true_positive
    true_negative = np.sum(confusion_matrix) - (true_positive + false_positive + false_negative)

    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    report.append({
        'Label': i,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    })

# Chuyển kết quả thành DataFrame
report_df = pd.DataFrame(report).set_index('Label')

print('Classification Report:')
print(report_df)


Confusion Matrix:
[[362.   0.   0.   0.   0.   0.]
 [  0. 376.   3.   0.   0.   0.]
 [  0.   2. 353.   0.   0.   0.]
 [  0.   0.   0. 340.   2.   0.]
 [  0.   0.   0.   0. 352.   3.]
 [  0.   0.   0.   0.   3. 259.]]
Classification Report:
       Precision    Recall  F1-Score
Label                               
0       1.000000  1.000000  1.000000
1       0.994709  0.992084  0.993395
2       0.991573  0.994366  0.992968
3       1.000000  0.994152  0.997067
4       0.985994  0.991549  0.988764
5       0.988550  0.988550  0.988550


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Khởi tạo các evaluator
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Tính toán Accuracy
accuracy = accuracy_evaluator.evaluate(predictions)
print('Accuracy:', accuracy)

# Tính toán F1 Score tổng hợp
f1 = f1_evaluator.evaluate(predictions)
print('F1 Score:', f1)


Accuracy: 0.9936739659367396
F1 Score: 0.9936781104954896


In [ ]:
from pyspark.ml.classification import RandomForestClassifier


In [ ]:
rf = RandomForestClassifier(featuresCol='scaledFeatures', labelCol='label', numTrees=100)

In [ ]:
rf_model = rf.fit(train_data)


In [ ]:
predictions_rf = rf_model.transform(test_data)

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy as np
import pandas as pd

# Tạo RDD chứa nhãn thật và nhãn dự đoán
predictionAndLabels = predictions_rf.select(col("prediction"), col("label")).rdd

# Khởi tạo MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabels)

# Tính toán confusion matrix
confusion_matrix = metrics.confusionMatrix().toArray()
print('Confusion Matrix:')
print(confusion_matrix)

# Lấy số lớp
num_classes = confusion_matrix.shape[0]

# Tính toán Precision, Recall và F1 Score cho từng lớp
report = []

for i in range(num_classes):
    true_positive = confusion_matrix[i, i]
    false_positive = np.sum(confusion_matrix[:, i]) - true_positive
    false_negative = np.sum(confusion_matrix[i, :]) - true_positive
    true_negative = np.sum(confusion_matrix) - (true_positive + false_positive + false_negative)

    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    report.append({
        'Label': i,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    })

# Chuyển kết quả thành DataFrame
report_df = pd.DataFrame(report).set_index('Label')

print('Classification Report:')
print(report_df)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion Matrix:
[[362.   0.   0.   0.   0.   0.]
 [  0. 360.  19.   0.   0.   0.]
 [  0.  11. 344.   0.   0.   0.]
 [  0.   0.   0. 333.   4.   5.]
 [  0.   0.   0.   5. 344.   6.]
 [  0.   0.   0.  12.  22. 228.]]
Classification Report:
       Precision    Recall  F1-Score
Label                               
0       1.000000  1.000000  1.000000
1       0.970350  0.949868  0.960000
2       0.947658  0.969014  0.958217
3       0.951429  0.973684  0.962428
4       0.929730  0.969014  0.948966
5       0.953975  0.870229  0.910180


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Khởi tạo các evaluator
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Tính toán Accuracy
accuracy = accuracy_evaluator.evaluate(predictions_rf)
print('Accuracy:', accuracy)

# Tính toán F1 Score tổng hợp
f1 = f1_evaluator.evaluate(predictions_rf)
print('F1 Score:', f1)

Accuracy: 0.9591240875912409
F1 Score: 0.958884303856698
